In [1]:
from edc import print_info 
print_info("geodb-manage")


***Notebook Title***  
GeoDB: Manage Datasets

***Notebook Description***  
Euro Data Cube GeoDB: Manage Datasets


***Notebook Dependencies***  
This notebook requires an active subscription to:
* EDC GeoDB


In [2]:
from edc import check_compatibility
check_compatibility("user-0.24.5", dependencies=["GEODB"])



---------

The following environment variables are available:

* `GEODB_AUTH_AUD`, `GEODB_AUTH_CLIENT_ID`, `GEODB_AUTH_DOMAIN`, `GEODB_API_SERVER_URL`, `GEODB_AUTH_CLIENT_SECRET`, `GEODB_API_SERVER_PORT`


## Manage Collections in your GeoDB



### Connecting to the GeoDB

In [3]:
from xcube_geodb.core.geodb import GeoDBClient

In [4]:
geodb = GeoDBClient()

In [5]:
# If you are logged in, this will tell you what account the system currently uses
geodb.whoami

'geodb_418dfeac-15f0-4606-9edb-fd9eb722bf04'

In [6]:
# Lets get already existing collections
ds = geodb.get_my_collections()
ds


,owner,database,collection,table_name
0,geodb_0b01bfcd-2d09-46f8-84e8-cb5720fba14c,geodb_0b01bfcd-2d09-46f8-84e8-cb5720fba14c,delineated_parcels_s,delineated_parcels_s
1,geodb_0b01bfcd-2d09-46f8-84e8-cb5720fba14c,geodb_0b01bfcd-2d09-46f8-84e8-cb5720fba14c,test_batic,test_batic
2,geodb_0d6df427-8c09-41b9-abc9-64ce13a68125,geodb_0d6df427-8c09-41b9-abc9-64ce13a68125,land_use,land_use
3,geodb_0d6df427-8c09-41b9-abc9-64ce13a68125,geodb_0d6df427-8c09-41b9-abc9-64ce13a68125,lpis_aut,lpis_aut
4,geodb_0e5d743f-2134-4561-8946-a073b039176f,geodb_0e5d743f-2134-4561-8946-a073b039176f,ai4eo_bboxes,ai4eo_bboxes
...,...,...,...,...
875,geodb_geodb_ci,geodb_geodb_ci,land_use,land_use
876,geodb_lpis_iacs_admin,lpis_iacs,land_use_slo,land_use_slo
877,geodb_lpis_iacs_admin,lpis_iacs,lpis_aut,lpis_aut
878,geodb_lpis_iacs_admin,lpis_iacs,lpis_slo,lpis_slo


In [7]:
ds[(ds.database == 'geodb_geodb_ci') & (ds.collection == 'land_use')].table_name.count() == 1

True

### Creating collections

Once the connection has been established you will be able to create a collection. The collection will contain standard properties (fields) plus custom properties
which you can add at your disgretion. Please use [PostGreSQL type definitions](https://www.postgresql.org/docs/11/datatype.html). We recommend stying simple with
your data types as we have not tested every single type.

In [8]:
ds = geodb.get_my_collections()

if ds[(ds.database == geodb.whoami) & (ds.collection == 'land_use')].collection.count() == 0:
    # Have a look at fiona feature schema
    collections = {
            "land_use": 
            {
                "crs": 3794,
                "properties": 
                {
                    "RABA_PID": "float", 
                    "RABA_ID": "float", 
                    "D_OD": "date"
                }
            }
        }


    geodb.create_collections(collections)
    import geopandas
    import os
    gdf = geopandas.read_file(os.path.expanduser("~/.shared/notebooks/eurodatacube/notebooks/getting-started/data/sample/land_use.shp"))
    geodb.insert_into_collection('land_use', gdf)

In [9]:
ds = geodb.get_my_collections()
ds

,owner,database,collection,table_name
0,geodb_0b01bfcd-2d09-46f8-84e8-cb5720fba14c,geodb_0b01bfcd-2d09-46f8-84e8-cb5720fba14c,delineated_parcels_s,delineated_parcels_s
1,geodb_0b01bfcd-2d09-46f8-84e8-cb5720fba14c,geodb_0b01bfcd-2d09-46f8-84e8-cb5720fba14c,test_batic,test_batic
2,geodb_0d6df427-8c09-41b9-abc9-64ce13a68125,geodb_0d6df427-8c09-41b9-abc9-64ce13a68125,land_use,land_use
3,geodb_0d6df427-8c09-41b9-abc9-64ce13a68125,geodb_0d6df427-8c09-41b9-abc9-64ce13a68125,lpis_aut,lpis_aut
4,geodb_0e5d743f-2134-4561-8946-a073b039176f,geodb_0e5d743f-2134-4561-8946-a073b039176f,ai4eo_bboxes,ai4eo_bboxes
...,...,...,...,...
875,geodb_geodb_ci,geodb_geodb_ci,land_use,land_use
876,geodb_lpis_iacs_admin,lpis_iacs,land_use_slo,land_use_slo
877,geodb_lpis_iacs_admin,lpis_iacs,lpis_aut,lpis_aut
878,geodb_lpis_iacs_admin,lpis_iacs,lpis_slo,lpis_slo


### Loading data into a dataset

Once the table has been created, you can load data into the dataset. The example below loads a shapefile. The attributes of the shapefile correspond to the dataset's properties.


In [10]:
import geopandas
import os
gdf = geopandas.read_file(os.path.expanduser("~/.shared/notebooks/eurodatacube/notebooks/getting-started/data/sample/land_use.shp"))
gdf

,RABA_PID,RABA_ID,D_OD,geometry
0,4770326.0,1410,2019-03-26,"POLYGON ((453952.629 91124.177, 453952.696 911..."
1,4770325.0,1300,2019-03-26,"POLYGON ((453810.376 91150.199, 453812.552 911..."
2,2305689.0,7000,2019-02-25,"POLYGON ((456099.635 97696.070, 456112.810 976..."
3,2305596.0,1100,2019-02-25,"POLYGON ((455929.405 97963.785, 455933.284 979..."
4,2310160.0,1100,2019-03-11,"POLYGON ((461561.512 96119.256, 461632.114 960..."
...,...,...,...,...
9822,6253989.0,1600,2019-03-08,"POLYGON ((460637.334 96865.891, 460647.927 969..."
9823,6252044.0,1600,2019-03-26,"POLYGON ((459467.868 96839.686, 459467.770 968..."
9824,6245985.0,2000,2019-04-08,"POLYGON ((459488.998 94066.248, 459498.145 940..."
9825,6245986.0,2000,2019-02-20,"POLYGON ((459676.680 94000.000, 459672.469 939..."


In [11]:
geodb.insert_into_collection('land_use', gdf)

Processing rows from 0 to 999


Processing rows from 1000 to 1999


Processing rows from 2000 to 2999


Processing rows from 3000 to 3999


Processing rows from 4000 to 4999


...

Processing rows from 6000 to 6999


Processing rows from 7000 to 7999


Processing rows from 8000 to 8999


Processing rows from 9000 to 9827


9827 rows inserted into land_use

In [12]:
geodb.get_collection('land_use', query="raba_id=eq.7000")

,id,created_at,modified_at,geometry,raba_pid,raba_id,d_od
0,3,2021-08-31T12:28:16.745304+00:00,None,"POLYGON ((456099.635 97696.070, 456112.810 976...",2305689,7000,2019-02-25
1,26,2021-08-31T12:28:16.745304+00:00,None,"POLYGON ((459898.930 100306.841, 459906.288 10...",2301992,7000,2019-04-06
2,95,2021-08-31T12:28:16.745304+00:00,None,"POLYGON ((459591.248 92619.056, 459592.745 926...",2333229,7000,2019-02-20
3,115,2021-08-31T12:28:16.745304+00:00,None,"POLYGON ((459013.303 100354.458, 459022.756 10...",2336738,7000,2019-03-15
4,129,2021-08-31T12:28:16.745304+00:00,None,"POLYGON ((460851.200 93442.039, 460846.405 934...",6292562,7000,2019-02-21
...,...,...,...,...,...,...,...
763,19246,2021-09-29T15:04:48.588108+00:00,None,"POLYGON ((456523.670 94000.000, 456550.368 939...",6187993,7000,2019-03-13
764,19373,2021-09-29T15:04:48.588108+00:00,None,"POLYGON ((460793.279 94000.000, 460797.466 939...",6219403,7000,2019-02-21
765,19387,2021-09-29T15:04:48.588108+00:00,None,"POLYGON ((461750.000 98434.292, 461750.000 984...",6219602,7000,2019-03-20
766,19451,2021-09-29T15:04:48.588108+00:00,None,"POLYGON ((463628.912 97501.218, 463636.859 975...",6215237,7000,2019-03-20


### Delete from a Collection

In [13]:
geodb.delete_from_collection('land_use', query="raba_id=eq.7000")

Data from land_use deleted

In [14]:
geodb.get_collection('land_use', query="raba_id=eq.7000")

,Empty Result


### Updating a Collection

In [15]:
geodb.get_collection('land_use', query="raba_id=eq.1300")

,id,created_at,modified_at,geometry,raba_pid,raba_id,d_od
0,2,2021-08-31T12:28:16.745304+00:00,None,"POLYGON ((453810.376 91150.199, 453812.552 911...",4770325,1300,2019-03-26
1,10,2021-08-31T12:28:16.745304+00:00,None,"POLYGON ((456547.427 91543.640, 456544.255 915...",2318555,1300,2019-03-14
2,63,2021-08-31T12:28:16.745304+00:00,None,"POLYGON ((456201.531 98685.274, 456199.109 986...",2304287,1300,2019-02-25
3,86,2021-08-31T12:28:16.745304+00:00,None,"POLYGON ((454709.766 97354.278, 454704.878 973...",2331038,1300,2019-01-05
4,87,2021-08-31T12:28:16.745304+00:00,None,"POLYGON ((453820.737 98574.017, 453816.740 985...",2357574,1300,2019-01-16
...,...,...,...,...,...,...,...
1785,19593,2021-09-29T15:04:48.588108+00:00,None,"POLYGON ((458247.583 99746.065, 458245.202 997...",6268059,1300,2019-03-04
1786,19594,2021-09-29T15:04:48.588108+00:00,None,"POLYGON ((461932.283 90520.703, 461931.619 905...",6263561,1300,2019-04-01
1787,19625,2021-09-29T15:04:48.588108+00:00,None,"POLYGON ((457231.170 90902.256, 457234.861 909...",6264862,1300,2019-01-05
1788,19626,2021-09-29T15:04:48.588108+00:00,None,"POLYGON ((461378.845 91124.318, 461490.224 911...",6264865,1300,2019-02-21


In [16]:
geodb.update_collection('land_use', query="raba_id=eq.1300", values={'d_od': '2000-01-01'})

land_use updated

In [17]:
geodb.get_collection('land_use', query="raba_id=eq.1300")

,id,created_at,modified_at,geometry,raba_pid,raba_id,d_od
0,10,2021-08-31T12:28:16.745304+00:00,2021-09-29T15:05:01.317443+00:00,"POLYGON ((456547.427 91543.640, 456544.255 915...",2318555,1300,2000-01-01
1,103,2021-08-31T12:28:16.745304+00:00,2021-09-29T15:05:01.317443+00:00,"POLYGON ((456100.880 96973.323, 456111.084 969...",2332420,1300,2000-01-01
2,108,2021-08-31T12:28:16.745304+00:00,2021-09-29T15:05:01.317443+00:00,"POLYGON ((454070.991 90528.946, 454064.259 905...",4674624,1300,2000-01-01
3,117,2021-08-31T12:28:16.745304+00:00,2021-09-29T15:05:01.317443+00:00,"POLYGON ((454637.692 96679.827, 454630.584 966...",2336101,1300,2000-01-01
4,136,2021-08-31T12:28:16.745304+00:00,2021-09-29T15:05:01.317443+00:00,"POLYGON ((462781.112 100486.430, 462803.178 10...",2334416,1300,2000-01-01
...,...,...,...,...,...,...,...
1785,19592,2021-09-29T15:04:48.588108+00:00,2021-09-29T15:05:01.317443+00:00,"POLYGON ((458304.555 100146.103, 458306.726 10...",6268058,1300,2000-01-01
1786,19593,2021-09-29T15:04:48.588108+00:00,2021-09-29T15:05:01.317443+00:00,"POLYGON ((458247.583 99746.065, 458245.202 997...",6268059,1300,2000-01-01
1787,19594,2021-09-29T15:04:48.588108+00:00,2021-09-29T15:05:01.317443+00:00,"POLYGON ((461932.283 90520.703, 461931.619 905...",6263561,1300,2000-01-01
1788,19625,2021-09-29T15:04:48.588108+00:00,2021-09-29T15:05:01.317443+00:00,"POLYGON ((457231.170 90902.256, 457234.861 909...",6264862,1300,2000-01-01


### Managing Properties of a Collection

In [18]:
geodb.get_my_collections()

,owner,database,collection,table_name
0,geodb_0b01bfcd-2d09-46f8-84e8-cb5720fba14c,geodb_0b01bfcd-2d09-46f8-84e8-cb5720fba14c,delineated_parcels_s,delineated_parcels_s
1,geodb_0b01bfcd-2d09-46f8-84e8-cb5720fba14c,geodb_0b01bfcd-2d09-46f8-84e8-cb5720fba14c,test_batic,test_batic
2,geodb_0d6df427-8c09-41b9-abc9-64ce13a68125,geodb_0d6df427-8c09-41b9-abc9-64ce13a68125,land_use,land_use
3,geodb_0d6df427-8c09-41b9-abc9-64ce13a68125,geodb_0d6df427-8c09-41b9-abc9-64ce13a68125,lpis_aut,lpis_aut
4,geodb_0e5d743f-2134-4561-8946-a073b039176f,geodb_0e5d743f-2134-4561-8946-a073b039176f,ai4eo_bboxes,ai4eo_bboxes
...,...,...,...,...
875,geodb_geodb_ci,geodb_geodb_ci,land_use,land_use
876,geodb_lpis_iacs_admin,lpis_iacs,land_use_slo,land_use_slo
877,geodb_lpis_iacs_admin,lpis_iacs,lpis_aut,lpis_aut
878,geodb_lpis_iacs_admin,lpis_iacs,lpis_slo,lpis_slo


In [19]:
geodb.get_properties('land_use')

,table_name,column_name,data_type


In [20]:
geodb.add_property('land_use', "test_prop", 'integer')

Properties added

In [21]:
geodb.get_properties('land_use')

,table_name,column_name,data_type


In [22]:
geodb.drop_property('land_use', 'test_prop')

Properties ['test_prop'] dropped from geodb_418dfeac-15f0-4606-9edb-fd9eb722bf04_land_use

In [23]:
geodb.get_properties('land_use')

,table_name,column_name,data_type


In [24]:
geodb.add_properties('land_use', properties={'test1': 'integer', 'test2': 'date'})

Properties added

In [25]:
geodb.get_properties('land_use')

,table_name,column_name,data_type


In [26]:
geodb.drop_properties('land_use', properties=['test1', 'test2'])

Properties ['test1', 'test2'] dropped from geodb_418dfeac-15f0-4606-9edb-fd9eb722bf04_land_use

In [27]:
geodb.get_properties('land_use')

,table_name,column_name,data_type


In [28]:
geodb.drop_collection('land_use')

Collection ['geodb_418dfeac-15f0-4606-9edb-fd9eb722bf04_land_use'] deleted